In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
%cd 'drive'

/content/drive


In [10]:
ls

'My Drive'/


In [11]:
%cd 'My Drive'

/content/drive/My Drive


In [14]:
%cd 'Data'

/content/drive/My Drive/Data


In [0]:
import numpy as np
import pandas as pd

In [0]:
df= pd.read_csv('train.csv')

In [17]:
df.head()

,user_name,country,review_title,review_description,designation,points,price,province,region_1,region_2,winery,variety
0,NaN,Australia,Andrew Peace 2007 Peace Family Vineyard Chardo...,"Classic Chardonnay aromas of apple, pear and h...",Peace Family Vineyard,83,10.0,Australia Other,South Eastern Australia,NaN,Andrew Peace,Chardonnay
1,@wawinereport,US,North by Northwest 2014 Red (Columbia Valley (...,This wine is near equal parts Syrah and Merlot...,NaN,89,15.0,Washington,Columbia Valley (WA),Columbia Valley,North by Northwest,Red Blend
2,NaN,Italy,Renato Ratti 2007 Conca (Barolo),Barolo Conca opens with inky dark concentratio...,Conca,94,80.0,Piedmont,Barolo,NaN,Renato Ratti,Nebbiolo
3,@vossroger,France,Domaine l'Ancienne Cure 2010 L'Abbaye White (B...,It's impressive what a small addition of Sauvi...,L'Abbaye,87,22.0,Southwest France,Bergerac Sec,NaN,Domaine l'Ancienne Cure,Bordeaux-style White Blend
4,@vossroger,France,Château du Cèdre 2012 Le Cèdre Vintage Malbec ...,"This ripe, sweet wine is rich and full of drie...",Le Cèdre Vintage,88,33.0,France Other,Vin de Liqueur,NaN,Château du Cèdre,Malbec


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82657 entries, 0 to 82656
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_name           63264 non-null  object 
 1   country             82622 non-null  object 
 2   review_title        82657 non-null  object 
 3   review_description  82657 non-null  object 
 4   designation         59010 non-null  object 
 5   points              82657 non-null  int64  
 6   price               77088 non-null  float64
 7   province            82622 non-null  object 
 8   region_1            69903 non-null  object 
 9   region_2            35949 non-null  object 
 10  winery              82657 non-null  object 
 11  variety             82657 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 7.6+ MB


In [19]:
df.describe()

,points,price
count,82657.000000,77088.000000
mean,88.544564,36.922232
std,3.099649,43.698346
min,80.000000,4.000000
25%,86.000000,18.000000
50%,88.000000,27.000000
75%,91.000000,45.000000
max,100.000000,3300.000000


In [20]:
#continuous data are points and price
df['points'].isnull().any()

False

In [21]:
df['price'].isnull().any()

True

In [0]:

#As we can see that price column has missing values.
#we generally replace any continuous data with mean value
df['price'].fillna((df['price'].mean()),inplace = True)

In [23]:
df['price'].isnull().any()

False

In [24]:
df['country'].isnull().any()

True

In [25]:

#As we know we need country label to fill in the missing values for province and regions,
#we needto find a solution to clean country first
df['country'].isnull().sum()

35

In [0]:
#From the above we can see that there are very few number of rows withcountries missing
#so let's take only those rows which are not null

df = df[df['country'].notnull()]

In [0]:
def points_to_class(points):
    if points in range(79,83):
        return 1
    elif points in range(83,87):
        return 2
    elif points in range(87,90):
        return 3
    elif points in range(90,94):
        return 4
    elif points in range(94,101):
        return 5
    
df["rating"] = df["points"].apply(points_to_class)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82622 entries, 0 to 82656
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_name           63229 non-null  object 
 1   country             82622 non-null  object 
 2   review_title        82622 non-null  object 
 3   review_description  82622 non-null  object 
 4   designation         58980 non-null  object 
 5   points              82622 non-null  int64  
 6   price               82622 non-null  float64
 7   province            82622 non-null  object 
 8   region_1            69903 non-null  object 
 9   region_2            35949 non-null  object 
 10  winery              82622 non-null  object 
 11  variety             82622 non-null  object 
 12  rating              82622 non-null  int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 8.8+ MB


In [0]:
def descriptions_to_word(rating):
  if rating == 1:
    return 'aroma'
  elif rating == 2:
    return 'finish'
  elif rating == 3:
    return 'note'
  elif rating == 4:
    return 'rich'
  else:
    return 'fruit'

In [0]:
df['one_word_review'] = df["rating"].apply(descriptions_to_word)

In [31]:
df.head()

,user_name,country,review_title,review_description,designation,points,price,province,region_1,region_2,winery,variety,rating,one_word_review
0,NaN,Australia,Andrew Peace 2007 Peace Family Vineyard Chardo...,"Classic Chardonnay aromas of apple, pear and h...",Peace Family Vineyard,83,10.0,Australia Other,South Eastern Australia,NaN,Andrew Peace,Chardonnay,2,finish
1,@wawinereport,US,North by Northwest 2014 Red (Columbia Valley (...,This wine is near equal parts Syrah and Merlot...,NaN,89,15.0,Washington,Columbia Valley (WA),Columbia Valley,North by Northwest,Red Blend,3,note
2,NaN,Italy,Renato Ratti 2007 Conca (Barolo),Barolo Conca opens with inky dark concentratio...,Conca,94,80.0,Piedmont,Barolo,NaN,Renato Ratti,Nebbiolo,5,fruit
3,@vossroger,France,Domaine l'Ancienne Cure 2010 L'Abbaye White (B...,It's impressive what a small addition of Sauvi...,L'Abbaye,87,22.0,Southwest France,Bergerac Sec,NaN,Domaine l'Ancienne Cure,Bordeaux-style White Blend,3,note
4,@vossroger,France,Château du Cèdre 2012 Le Cèdre Vintage Malbec ...,"This ripe, sweet wine is rich and full of drie...",Le Cèdre Vintage,88,33.0,France Other,Vin de Liqueur,NaN,Château du Cèdre,Malbec,3,note


In [32]:
df['country'].value_counts()

US                38573
France            15948
Italy              9112
Portugal           3197
Chile              2891
Spain              2755
Argentina          2570
Austria            1892
Germany            1573
New Zealand        1121
Australia          1028
South Africa        727
Israel              320
Canada              176
Greece              139
Bulgaria             93
Romania              80
England              62
Slovenia             42
Mexico               38
Hungary              35
Moldova              35
Turkey               33
Uruguay              33
Lebanon              28
Brazil               28
Morocco              21
Georgia              12
Croatia              12
Ukraine              11
Peru                  8
Cyprus                6
Macedonia             5
Switzerland           5
Luxembourg            5
Serbia                4
Czech Republic        3
India                 1
Name: country, dtype: int64

In [0]:

s = df['country'].value_counts()
df['country'] = np.where(df['country'].isin(s.index[s >= 2000]), df['country'], 'Other')

In [34]:
df['country'].value_counts()

US           38573
France       15948
Italy         9112
Other         7576
Portugal      3197
Chile         2891
Spain         2755
Argentina     2570
Name: country, dtype: int64

In [35]:
df['winery'].value_counts()

Testarossa               175
Louis Latour             168
Williams Selyem          165
Chateau Ste. Michelle    163
Georges Duboeuf          163
                        ... 
Pugilist                   1
Abad                       1
Château de Bergun          1
Cerebella                  1
Inkling                    1
Name: winery, Length: 13776, dtype: int64

In [0]:
s = df['winery'].value_counts()
df['winery'] = np.where(df['winery'].isin(s.index[s >= 90]), df['winery'], 'Other')

In [37]:
df['winery'].value_counts()

Other                         80620
Testarossa                      175
Louis Latour                    168
Williams Selyem                 165
Chateau Ste. Michelle           163
Georges Duboeuf                 163
Wines & Winemakers              142
DFJ Vinhos                      131
Concha y Toro                   112
Columbia Crest                  112
Kendall-Jackson                 100
Siduri                           99
Lynmar                           98
Gary Farrell                     98
Albert Bichot                    94
Jean-Luc and Paul Aegerter       92
Montes                           90
Name: winery, dtype: int64

In [41]:
df['province'].value_counts()

California        25736
Washington         6060
Bordeaux           4690
Oregon             3900
Tuscany            3780
                  ...  
Portugal              1
Santa Catarina        1
Ceres Plateau         1
Távora-Varosa         1
Dan                   1
Name: province, Length: 358, dtype: int64

In [0]:
s = df['province'].value_counts()
df['province'] = np.where(df['province'].isin(s.index[s >= 3500]), df['province'], 'Other')

In [65]:
df['region_1'].value_counts()

Napa Valley                3334
Columbia Valley (WA)       2951
Russian River Valley       2362
California                 1902
Willamette Valley          1739
                           ... 
Coteaux d'Ancenis             1
Barossa-Adelaide Hills        1
Vin de Pays de France         1
Passito di Pantelleria        1
Montepulciano d'Abruzzo       1
Name: region_1, Length: 1019, dtype: int64

In [0]:
s = df['region_1'].value_counts()
df['region_1'] = np.where(df['region_1'].isin(s.index[s >= 1000]), df['region_1'], 'Other')

In [71]:
df['region_1'].value_counts().sum() #nan also gets replaced by other

82622

In [84]:
df['region_2'].value_counts()

Central Coast        7503
Sonoma               6776
Columbia Valley      5679
Napa                 5119
Willamette Valley    2605
California Other     1926
Finger Lakes         1314
Napa-Sonoma           919
Sierra Foothills      772
Central Valley        641
Southern Oregon       548
Oregon Other          519
Long Island           503
North Coast           411
Washington Other      380
South Coast           173
New York Other        161
Name: region_2, dtype: int64

In [0]:
s = df['region_2'].value_counts()
df['region_2'] = np.where(df['region_2'].isin(s.index[s >= 700]), df['region_2'], 'Other')

In [98]:
df['designation'].value_counts()

Reserve                                 1380
Estate                                   898
Reserva                                  743
Estate Grown                             425
Riserva                                  415
                                        ... 
Old Hat                                    1
Estate St. Helena Vineyard Rosato Di       1
Bianco del Mulino                          1
Quinta do Cachão                           1
Schiaccianoci                              1
Name: designation, Length: 26407, dtype: int64

In [0]:
s = df['designation'].value_counts()
df['designation'] = np.where(df['designation'].isin(s.index[s >= 300]), df['designation'], 'Other')

In [102]:
df['designation'].value_counts()

Other            78458
Reserve           1380
Estate             898
Reserva            743
Estate Grown       425
Riserva            415
Barrel sample      303
Name: designation, dtype: int64

MODEL BUILDING


In [0]:
X= df[['price','rating','one_word_review','country','province','points','region_1','region_2','designation']]
#for price,rating,onewordreview,country,province,points ---> 34.45 accuracy
#for price,rating,onewordreview,country,province,points,region_1 ---> 37.5 accuracy
#for price,rating,onewordreview,country,province,points,region_1,'region_2' ---> 38.8 accuracy
#for price,rating,onewordreview,country,province,points,region_1,'region_2','designation' ---> 39 accuracy

In [104]:
X.head()

,price,rating,one_word_review,country,province,points,region_1,region_2,designation
0,10.0,2,finish,Other,Other,83,Other,Other,Other
1,15.0,3,note,US,Washington,89,Columbia Valley (WA),Columbia Valley,Other
2,80.0,5,fruit,Italy,Other,94,Barolo,Other,Other
3,22.0,3,note,France,Other,87,Other,Other,Other
4,33.0,3,note,France,Other,88,Other,Other,Other


In [105]:
X_2 =X
X_2

,price,rating,one_word_review,country,province,points,region_1,region_2,designation
0,10.0,2,finish,Other,Other,83,Other,Other,Other
1,15.0,3,note,US,Washington,89,Columbia Valley (WA),Columbia Valley,Other
2,80.0,5,fruit,Italy,Other,94,Barolo,Other,Other
3,22.0,3,note,France,Other,87,Other,Other,Other
4,33.0,3,note,France,Other,88,Other,Other,Other
...,...,...,...,...,...,...,...,...,...
82652,12.0,2,finish,Spain,Other,84,Other,Other,Other
82653,21.0,2,finish,US,Other,85,Other,Other,Other
82654,14.0,3,note,France,Other,87,Other,Other,Other
82655,20.0,4,rich,France,Bordeaux,91,Other,Other,Other


In [0]:
X_2 = pd.get_dummies(X_2, columns=['one_word_review','country','province','region_1','region_2','designation'])


In [107]:

X_2 = X_2.iloc[:,:].values
X_2


array([[10.,  2., 83., ...,  0.,  0.,  0.],
       [15.,  3., 89., ...,  0.,  0.,  0.],
       [80.,  5., 94., ...,  0.,  0.,  0.],
       ...,
       [14.,  3., 87., ...,  0.,  0.,  0.],
       [20.,  4., 91., ...,  0.,  0.,  0.],
       [50.,  4., 92., ...,  0.,  0.,  0.]])

In [108]:
y = df['variety']
#let us do the label encoding to our variety variable which is the target
from sklearn.preprocessing import LabelEncoder
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)
y

array([ 5, 17, 11, ...,  6,  0,  3])

In [0]:

from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X_2, y, test_size = 0.2, random_state = 0)

In [0]:
from sklearn.ensemble import RandomForestClassifier

regressor = RandomForestClassifier(n_estimators=20, random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_cv)

In [111]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_cv,y_pred))
print(classification_report(y_cv,y_pred))
print(accuracy_score(y_cv, y_pred))

[[ 771   20    4   92    0   23    5    0    2   16   10    0    0    1
    25    3    0   26    6   32   19    2    6    2   35    3    6    5]
 [ 137   11    1    2    0    3    1    0    0    0    1    0    0    0
     1    0    0    1    0    0    6    0    0    0    0    0    0    0]
 [  13    0   16   36    1   29    2    1    1   10   13    0    0    0
    26    0    0   13    0   18   16    1   10    2   19    0    1    4]
 [  64    0    8  652    0  121    0    4    8   64   66    5    9    2
    95    1    0   90    5   36   17   13   79    9   77   12   12   76]
 [   0    0    0    3  167   17    2    0    0    1    0    0    0    0
     8    0    0    2    0    4    2    0    1    0    0    1    0    0]
 [  19    1    5  138   42  578   19   10   14   41   53   12   19   12
   426    2    0   54   13  112   94    1  105   20   38    9   19   37]
 [   0    0    0    0    0   28   29    0    0    4    0    0    0    0
    12    0    0    4    4    0   65    0    5    2    0  

In [0]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [0]:
nb.fit(X_train, y_train)
y_pred = regressor.predict(X_cv)

In [114]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_cv,y_pred))
print(classification_report(y_cv,y_pred))
print(accuracy_score(y_cv, y_pred))

[[ 771   20    4   92    0   23    5    0    2   16   10    0    0    1
    25    3    0   26    6   32   19    2    6    2   35    3    6    5]
 [ 137   11    1    2    0    3    1    0    0    0    1    0    0    0
     1    0    0    1    0    0    6    0    0    0    0    0    0    0]
 [  13    0   16   36    1   29    2    1    1   10   13    0    0    0
    26    0    0   13    0   18   16    1   10    2   19    0    1    4]
 [  64    0    8  652    0  121    0    4    8   64   66    5    9    2
    95    1    0   90    5   36   17   13   79    9   77   12   12   76]
 [   0    0    0    3  167   17    2    0    0    1    0    0    0    0
     8    0    0    2    0    4    2    0    1    0    0    1    0    0]
 [  19    1    5  138   42  578   19   10   14   41   53   12   19   12
   426    2    0   54   13  112   94    1  105   20   38    9   19   37]
 [   0    0    0    0    0   28   29    0    0    4    0    0    0    0
    12    0    0    4    4    0   65    0    5    2    0  

In [0]:
# TO increase the accuracy I am adding two more "one word review" for the "description review"

In [0]:
def descriptions_to_word_2(rating):
  if rating == 1:
    return 'finish'
  elif rating == 2:
    return 'aroma'
  elif rating == 3:
    return 'fresh'
  elif rating == 4:
    return 'black cherry'
  else:
    return 'acidity'

In [0]:
def descriptions_to_word_3(rating):
  if rating == 1:
    return 'palate'
  elif rating == 2:
    return 'soft'
  elif rating == 3:
    return 'rich'
  elif rating == 4:
    return 'full bodied'
  else:
    return 'ripe'

In [0]:
df['one_word_review_2'] = df["rating"].apply(descriptions_to_word_2)

In [0]:
df['one_word_review_3'] = df["rating"].apply(descriptions_to_word_3)

In [124]:
df.head()

,user_name,country,review_title,review_description,designation,points,price,province,region_1,region_2,winery,variety,rating,one_word_review,one_word_review_2,one_word_review_3
0,NaN,Other,Andrew Peace 2007 Peace Family Vineyard Chardo...,"Classic Chardonnay aromas of apple, pear and h...",Other,83,10.0,Other,Other,Other,Other,Chardonnay,2,finish,aroma,soft
1,@wawinereport,US,North by Northwest 2014 Red (Columbia Valley (...,This wine is near equal parts Syrah and Merlot...,Other,89,15.0,Washington,Columbia Valley (WA),Columbia Valley,Other,Red Blend,3,note,fresh,rich
2,NaN,Italy,Renato Ratti 2007 Conca (Barolo),Barolo Conca opens with inky dark concentratio...,Other,94,80.0,Other,Barolo,Other,Other,Nebbiolo,5,fruit,acidity,ripe
3,@vossroger,France,Domaine l'Ancienne Cure 2010 L'Abbaye White (B...,It's impressive what a small addition of Sauvi...,Other,87,22.0,Other,Other,Other,Other,Bordeaux-style White Blend,3,note,fresh,rich
4,@vossroger,France,Château du Cèdre 2012 Le Cèdre Vintage Malbec ...,"This ripe, sweet wine is rich and full of drie...",Other,88,33.0,Other,Other,Other,Other,Malbec,3,note,fresh,rich


In [0]:
X= df[['price','rating','one_word_review','one_word_review_2','one_word_review_3','country','province','region_1','region_2','points']]


In [216]:
X_2 =X
X_2

,price,rating,one_word_review,one_word_review_2,one_word_review_3,country,province,region_1,region_2,points
0,10.0,2,finish,aroma,soft,Other,Other,Other,Other,83
1,15.0,3,note,fresh,rich,US,Washington,Columbia Valley (WA),Columbia Valley,89
2,80.0,5,fruit,acidity,ripe,Italy,Other,Barolo,Other,94
3,22.0,3,note,fresh,rich,France,Other,Other,Other,87
4,33.0,3,note,fresh,rich,France,Other,Other,Other,88
...,...,...,...,...,...,...,...,...,...,...
82652,12.0,2,finish,aroma,soft,Spain,Other,Other,Other,84
82653,21.0,2,finish,aroma,soft,US,Other,Other,Other,85
82654,14.0,3,note,fresh,rich,France,Other,Other,Other,87
82655,20.0,4,rich,black cherry,full bodied,France,Bordeaux,Other,Other,91


In [0]:
X_2 = pd.get_dummies(X_2, columns=['one_word_review','one_word_review_2','one_word_review_3','country','province','region_1','region_2'])

In [0]:

from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X_2, y, test_size = 0.2, random_state = 0)

In [0]:
from sklearn.ensemble import RandomForestClassifier

regressor = RandomForestClassifier(n_estimators=20, random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_cv)

In [222]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_cv,y_pred))
print(classification_report(y_cv,y_pred))
print(accuracy_score(y_cv, y_pred))

[[ 773   15    6   95    0   22    5    1    2   16   12    0    0    2
    21    3    0   29    6   31   18    3    5    2   32    3    6    6]
 [ 139    9    1    3    0    2    1    0    0    0    0    0    0    0
     1    0    0    1    0    0    6    0    0    0    1    0    0    0]
 [  14    0   11   38    1   32    2    1    1    9   12    0    0    0
    26    0    0   14    0   19   15    0   10    2   17    1    1    6]
 [  62    0   13  656    0  121    0    4    7   62   59    3    9    2
    96    2    0   92    7   40   18   13   80   12   74   10   12   71]
 [   0    0    0    2  167   17    2    0    0    1    0    0    0    0
     9    0    0    2    0    4    1    0    2    0    0    1    0    0]
 [  18    1    5  135   43  584   19   10   15   40   48   12   20   13
   424    2    0   53   17  112   90    1  117   17   33   10   18   36]
 [   0    0    0    0    0   28   29    0    0    4    0    0    0    0
    12    0    0    4    4    0   65    0    5    2    0  